In [114]:
import data_functions as dat
import importlib
import itertools
import math
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import pyDOE as pyd
import scipy.optimize as sco
import setup_analysis as sa
import time
import ri_water_model as wm
importlib.reload(dat)
importlib.reload(sa)

<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/dmdu_scalable_computing_2022/crdm_project/python/setup_analysis.py'>

###  First, let's read in key data sets and print off some information for our analysis.

In [109]:
# read in input data
df_model_data = pd.read_csv(sa.fp_csv_baseline_trajectory_model_input_data)
df_climate_deltas_annual = pd.read_csv(sa.fp_csv_climate_deltas_annual)

# show a record of info
print(f"Analysis Information from setup_analysis.py\n\n\tanalysis_name:\t{sa.analysis_name}\n\tn_lhs:\t{sa.n_lhs}")


Analysis Information from setup_analysis.py

	analysis_name:	crdm_project_2021091
	n_lhs:	1000


###  Next, let's load the strategy table

In [124]:
# get the strategies and attribute table from the excel file
df_attr_strategy, df_strategies = dat.get_strategy_table(sa.fp_xlsx_strategy_inputs)


###  Add the LHS samples

In [ ]:

# setup ranges for lhs
dict_ranges = {
    "flow_m3s": [
        0.95*min(df_climate_deltas_annual["delta_q_2055_annual"]), 
        1.05*max(df_climate_deltas_annual["delta_q_2055_annual"])
    ],
    "precipitation_mm": [
        0.95*min(df_climate_deltas_annual["delta_p_2055_annual"]), 
        1.05*max(df_climate_deltas_annual["delta_p_2055_annual"])
    ],
    "population": [0.8, 1.3],
    "demand_municipal_m3p": [1, 1],
    "demand_agricultural_m3km2": [1, 1],
    "area_ag_km2": [0.8, 1.5]
}
# set future 0 values - different because we'll apply deltas differently
dict_f0_vals = {
    "flow_m3s": 0,
    "precipitation_mm": 0,
    "population": 1,
    "demand_municipal_m3p": 1,
    "demand_agricultural_m3km2": 1,
    "area_ag_km2": 1
}
# build lhs samples
df_lhs = dat.generate_lhs_samples(sa.n_lhs, dict_ranges, dict_f0_vals)

###  Apply the climate deltas

In [ ]:

##  Climate Deltas

t0 = time.time()
dict_climate_factor_delta_field_map =  {
    "flow_m3s": "flow_m3s",
    "precipitation_mm": "precipitation_mm"
}
df_climate_deltas_by_future = dat.get_climate_factor_deltas(
    df_model_data,
    df_lhs,
    dict_climate_factor_delta_field_map,
    sa.range_delta_base, 
    sa.range_delta_fut, 
    max(sa.model_historical_years)
)
t1 = time.time()
t_del = np.round(t1 - t0, 2)
print(f"Time to generate {sa.n_lhs} climate deltas: {t_del} seconds")



###  Let's add the other uncertainty deltas (treated differently)

In [133]:
# get time period information
t0 = max(df_model_data[df_model_data["year"] == min(sa.model_projection_years) - 1]["time_period"])
t1 = max(df_model_data["time_period"])

# run the function
df_other_deltas_by_future = dat.get_linear_delta_trajectories_by_future(
    df_model_data,
    df_lhs[[x for x in df_lhs.columns if x not in dict_climate_factor_delta_field_map.keys()]],
    t0,
    t1
)
# merge back in some data
df_other_deltas_by_future = pd.merge(df_other_deltas_by_future, df_model_data[["time_period", "year", "month"]])



###  Build a unified futures database

In [150]:
df_futures = pd.merge(df_climate_deltas_by_future, df_other_deltas_by_future)
# some column sorting
fields_ind = ["future_id", "time_period", "year", "month"]
fields_dat = sorted([x for x in df_futures.columns if x not in fields_ind])
df_futures = df_futures[fields_ind + fields_dat]

###  Create a primary key, which we can use for an experimental design

In [151]:
# create a primary key
fields_index = ["strategy_id", "future_id"]
field_primary_key = "primary_id"
df_attribute_primary = pd.DataFrame(
    list(itertools.product(
        list(df_attr_strategy["strategy_id"]), 
        list(df_lhs["future_id"])
    )), 
    columns = fields_index
)
df_attribute_primary[field_primary_key] = range(len(df_attribute_primary))
df_attribute_primary = df_attribute_primary[[field_primary_key] + fields_index]

###  Run the model under default conditions

In [143]:
df_out = wm.ri_water_resources_model(df_model_data)

RI model done in 7.57 seconds.


###  Define a function to run the model for a single scenario

In [159]:
# function here
def run_model_by_scenario(
    row_scenario: pd.DataFrame,
    df_futures: pd.DataFrame,
    df_strategies: pd.DataFrame,
    field_primary_key: str = "primary_id",
    field_future: str = "future_id", 
    field_strategy: str = "strategy_id"
):
    # get ids
    id_future = int(row_scenario[field_future])
    id_primary = int(row_scenario[field_primary_key])
    id_strategy = int(row_scenario[field_strategy])
    
    # get input data
    df_future = df_futures[df_futures[field_future] == id_future].copy()
    df_future.drop([field_future], axis = 1, inplace = True)
    df_strategy = df_strategies[df_strategies[field_strategy] == id_strategy].copy()
    df_strategy.drop([field_strategy], axis = 1, inplace = True)
    df_input_data = pd.merge(df_future, df_strategy)
    
    # model output
    df_out = wm.ri_water_resources_model(df_input_data)
    fields_ord = list(df_out.columns)
    df_out[field_primary_key] = id_primary
    df_out = df_out[[field_primary_key] + fields_ord]
    
    return df_out



###  Try running the model for a selected primary_id

In [ ]:
id_primary = 474
run_model_by_scenario(
    df_attribute_primary[df_attribute_primary[field_primary_key] == id_primary], 
    df_futures,
    df_strategies
)